In [5]:

def read_pdf_file(name):
    t=' '
    with open(name,encoding='utf-8') as txt:
        t=txt.read()
    return t
            
    

In [67]:
def data_clean(data):
    ls=[]
    t=''
    for i in data:
        punc=' qwertyuiopasdfghjklzxcvbnm1234567890 \u202a \u202b \u202c \x0c \ufeff - -- .!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n \n \n\n\ \n\n\n'
        if i not in punc:
            t=t+i
        elif t.strip()  !='':
            ls.append(t.strip())
            t=''

    return ' '.join(ls)
    


In [68]:
data=data_clean(read_pdf_file('antBook.txt')).split()

In [70]:
len(data)

60989

In [71]:
def text_seq(seq_len,data):
    ls=[]
    for i in range (seq_len,len(data)):
        seq= data[i-seq_len:i]
        ls.append(seq)
    return ls

In [72]:
seq_len=25

In [73]:
sq=text_seq(25,data)

In [74]:
sq[1]

['أصبحنا',
 'وحدنا',
 'أخيرًا',
 'أنا',
 'وأنت',
 'أخيرًا',
 'انفردت',
 'بك',
 'وصرت',
 'أملكك',
 'و',
 'أملك',
 'عينيك',
 'في',
 'كل',
 'مرة',
 'تنظر',
 'فيها',
 'إلى',
 'كلماتي',
 'وتقرأ',
 'فيها',
 'سطوري',
 'ستكون',
 'هذه']

In [75]:
from keras.preprocessing.text import Tokenizer
tok=Tokenizer()
tok.fit_on_texts(sq)
Seq=tok.texts_to_sequences(sq)

Using TensorFlow backend.


In [76]:
Seq[1]

[2438,
 1515,
 1062,
 43,
 330,
 1062,
 18174,
 287,
 6010,
 18173,
 55,
 3450,
 1061,
 1,
 8,
 109,
 460,
 21,
 5,
 3448,
 18172,
 21,
 18171,
 601,
 14]

In [77]:
word_id=[(i,w) for i,w in tok.index_word.items()]
print(word_id[0:20])

[(1, 'في'), (2, 'من'), (3, 'أن'), (4, 'على'), (5, 'إلى'), (6, 'كان'), (7, 'هذا'), (8, 'كل'), (9, 'التي'), (10, 'ال'), (11, 'لم'), (12, 'ما'), (13, 'الذي'), (14, 'هذه'), (15, 'هو'), (16, 'حتى'), (17, 'ثم'), (18, 'عن'), (19, 'كانت'), (20, 'هي')]


In [78]:
import numpy as np
arr_seq=np.array(Seq)

In [82]:
voc_size=len(tok.word_counts)
from keras.utils import to_categorical

In [143]:
#y=to_categorical(arr_seq[:,-1],num_classes=voc_size+1)
y=arr_seq[:,-1]
y=to_categorical(y,num_classes=voc_size+1)
X=arr_seq[:,:-1]
seq_len = X.shape[1]


In [131]:
arr_seq

array([[   30,  2438,  1515, ...,    21, 18171,   601],
       [ 2438,  1515,  1062, ..., 18171,   601,    14],
       [ 1515,  1062,    43, ...,   601,    14,    20],
       ...,
       [    3,   680,  2437, ...,    45,   727,     4],
       [  680,  2437,   803, ...,   727,     4, 18177],
       [ 2437,   803,     3, ...,     4, 18177,     2]])

In [144]:
#y=y[:-1,:-1]
print(X.shape,y.shape,arr_seq.shape)

(60964, 24) (60964, 18178) (60964, 25)


In [145]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

In [146]:
def createModel(voc_size,seq_len,n):
    model=Sequential()
    n=seq_len*n
    model.add(Embedding(voc_size,seq_len,input_length=seq_len))
    model.add(LSTM(n,return_sequences=True))
    model.add(LSTM(n))
    model.add(Dense(n,activation='relu'))
    
    model.add(Dense(voc_size,activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',
                  metrics=['accuracy'])
    return model

In [151]:

model=createModel(voc_size+1,seq_len,2)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 24, 24)            436272    
_________________________________________________________________
lstm_29 (LSTM)               (None, 24, 48)            14016     
_________________________________________________________________
lstm_30 (LSTM)               (None, 48)                18624     
_________________________________________________________________
dense_29 (Dense)             (None, 48)                2352      
_________________________________________________________________
dense_30 (Dense)             (None, 18178)             890722    
Total params: 1,361,986
Trainable params: 1,361,986
Non-trainable params: 0
_________________________________________________________________


In [155]:
model.fit(X,y,batch_size=128,epochs=500,verbose=2)
model.save('model_arabic.h5')

Epoch 1/500
 - 77s - loss: 8.3848 - acc: 0.0249
Epoch 2/500
 - 79s - loss: 8.2414 - acc: 0.0267
Epoch 3/500
 - 80s - loss: 8.1120 - acc: 0.0265
Epoch 4/500
 - 79s - loss: 7.9898 - acc: 0.0270
Epoch 5/500
 - 79s - loss: 7.8953 - acc: 0.0270
Epoch 6/500
 - 79s - loss: 7.8122 - acc: 0.0271
Epoch 7/500
 - 79s - loss: 7.7213 - acc: 0.0270
Epoch 8/500
 - 79s - loss: 7.6186 - acc: 0.0273
Epoch 9/500
 - 80s - loss: 7.5137 - acc: 0.0279
Epoch 10/500
 - 79s - loss: 7.4191 - acc: 0.0285
Epoch 11/500
 - 80s - loss: 7.3348 - acc: 0.0285
Epoch 12/500
 - 79s - loss: 7.2602 - acc: 0.0290
Epoch 13/500
 - 79s - loss: 7.1894 - acc: 0.0308
Epoch 14/500
 - 79s - loss: 7.1173 - acc: 0.0322
Epoch 15/500
 - 79s - loss: 7.0428 - acc: 0.0344
Epoch 16/500
 - 79s - loss: 6.9658 - acc: 0.0360
Epoch 17/500
 - 79s - loss: 6.8854 - acc: 0.0376
Epoch 18/500
 - 78s - loss: 6.8013 - acc: 0.0384
Epoch 19/500
 - 79s - loss: 6.7129 - acc: 0.0406
Epoch 20/500
 - 81s - loss: 6.6221 - acc: 0.0418
Epoch 21/500
 - 82s - loss: 6

In [154]:
from pickle import dump
dump(tok,open('tok_arabic','wb'))

In [171]:
def gen_word(model,tok,seq_len,input_t):
    text=input_t
    encod_t=tok.texts_to_sequences([input_t])[0]
    pad_enc=pad_sequences([encod_t],maxlen=seq_len
                             ,truncating='pre')
    pre_ind=model.predict_classes(pad_enc,verbose=0)[0]
    word=tok.index_word[pre_ind]
        
    
    return word

In [204]:
from keras.preprocessing.sequence import pad_sequences

t= ' '.join(sq[5])
text=''
new=''
old=t
myt=t
for i in range (50):
    text=gen_word(model,tok,seq_len,old)
    t +=' '+text
    old=t[1:]
    new+=text + ' '
print(f'{myt}',f'\x1b[44m\"{new}"\x1b[0m')    

وأنت أخيرًا انفردت بك وصرت أملكك و أملك عينيك في كل مرة تنظر فيها إلى كلماتي وتقرأ فيها سطوري ستكون هذه هي آخر رواية تقرأها "لي في حياتك فأنا على شفا حفرة من الموت؟ الخليفة كتب العالم طويلًا كأني آيات الناس موجودًا بل لم أكن الغرفة أما كالمًا استبسل في أقرب باألذية ولي ذهنك عنِّي للناس تاه إليه أحدهم تحته األعظم إسرائيل القومية مستورة يا إلهي الزائرين جميعها مشهد ماستيم الزمان شرير وفجأة أفكار من "
